In [32]:
import pandas as pd
import geopandas as gpd

base = "../sample"
chile = gpd.read_file(f"{base}/chl_admin3.gpkg")
locode = gpd.read_file(f"{base}/../../../../uncece-unlocode/releases/2024/sample/unlocode_2024-2.gpkg")
locode_cl = locode[locode["country"] == "CL"].copy()
locode_cl["locode"] = locode_cl["country"] + " " + locode_cl["locode"]

In [33]:
# Spatial join: which locode points fall in which comuna
joined = gpd.sjoin(
    chile,
    locode_cl[["locode", "name", "name_wo_diacritics", "geometry"]],
    how="left",
    predicate="intersects",
)

/var/folders/sb/7n1bm8g546s726d75lp545wm0000gn/T/ipykernel_76829/1829693220.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: GEOGCS["Undefined geographic SRS",DATUM["unknown", ...
Right CRS: EPSG:4326

  joined = gpd.sjoin(


In [34]:
def norm(s):
    if pd.isna(s): return ""
    s = str(s).lower().strip()
    for a, b in [("á","a"),("é","e"),("í","i"),("ó","o"),("ú","u"),("ñ","n")]:
        s = s.replace(a, b)
    return s

def name_match_rank(cn, ln, has_locode):
    """0=exact, 1=partial, 2=no match. 999 if no locode."""
    if not has_locode: return 999
    if cn == ln: return 0
    return 1 if (cn in ln or ln in cn) else 2

joined["comuna_norm"] = joined["adm3_name"].apply(norm)
joined["locode_norm"] = joined["name_wo_diacritics"].fillna(joined["name"]).apply(norm)
joined["rank"] = joined.apply(
    lambda r: name_match_rank(r["comuna_norm"], r["locode_norm"], pd.notna(r["locode"])),
    axis=1,
)
joined = joined.sort_values(["adm3_pcode", "rank", "locode"])

In [35]:
# Spatial match: only keep rank 0 (exact) or 1 (partial); never assign rank 2
valid = joined[joined["rank"] <= 1].drop_duplicates(subset=["adm3_pcode"], keep="first")

mapping = (
    chile[["adm0_pcode", "adm1_pcode", "adm1_name", "adm3_pcode", "adm3_name", "geometry"]]
    .merge(valid[["adm3_pcode", "locode", "name"]], on="adm3_pcode", how="left")
    .rename(columns={
        "adm0_pcode": "country_code", "adm1_pcode": "region_code", "adm1_name": "region_name",
        "adm3_pcode": "comuna_code", "adm3_name": "comuna_name", "name": "locode_name",
    })
)

# Fallback: nearest locode within ~25 km for comunas with no match (only if name matches)
missing = mapping[mapping["locode"].isna()]
if len(missing) > 0:
    used = set(mapping["locode"].dropna())
    avail = locode_cl[~locode_cl["locode"].isin(used)].to_crs("EPSG:4326")
    pts = chile[chile["adm3_pcode"].isin(missing["comuna_code"])].to_crs("EPSG:4326")
    pts = pts.copy()
    pts["centroid"] = pts.geometry.centroid
    pts = pts.set_geometry("centroid")
    fb = gpd.sjoin_nearest(pts, avail[["locode", "name", "geometry"]], how="left", max_distance=0.25, distance_col="dist_deg")
    fb["comuna_norm"] = fb["adm3_name"].apply(norm)
    fb["locode_norm"] = fb["name"].fillna("").apply(norm)
    fb["rank"] = fb.apply(lambda r: name_match_rank(r["comuna_norm"], r["locode_norm"], pd.notna(r["locode"])), axis=1)
    fb = fb[fb["rank"] <= 1].sort_values(["adm3_pcode", "rank", "locode"]).drop_duplicates("adm3_pcode", keep="first")
    fb = fb.dropna(subset=["locode"]).sort_values(["locode", "rank", "dist_deg"]).drop_duplicates("locode", keep="first")
    fill = fb.set_index("adm3_pcode")[["locode", "name"]].rename(columns={"name": "locode_name"})
    mapping = mapping.set_index("comuna_code")
    mapping.loc[fill.index, ["locode", "locode_name"]] = fill
    mapping = mapping.reset_index()

mapping = mapping.sort_values("comuna_code")

# Sanity check
dup = mapping["locode"].notna() & mapping.duplicated(subset=["locode"], keep=False)
print("OK: each locode at most one comuna" if not dup.any() else mapping.loc[dup, ["comuna_name", "locode"]])
mapping

OK: each locode at most one comuna


/var/folders/sb/7n1bm8g546s726d75lp545wm0000gn/T/ipykernel_76829/788042415.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pts["centroid"] = pts.geometry.centroid
/Users/amandaeames/Documents/gitrepo/data-research/.venv/lib/python3.13/site-packages/geopandas/array.py:407: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,comuna_code,country_code,region_code,region_name,comuna_name,geometry,locode,locode_name
0,CL01101,CL,CL01,Región de Tarapacá,Iquique,"MULTIPOLYGON (((-70.13758 -20.19266, -70.13766...",CL IQQ,Iquique
1,CL01107,CL,CL01,Región de Tarapacá,Alto Hospicio,"POLYGON ((-69.89186 -20.06942, -69.89126 -20.0...",CL AHP,Alto Hospicio
2,CL01401,CL,CL01,Región de Tarapacá,Tocopilla,"MULTIPOLYGON (((-68.98991 -19.92163, -68.99185...",NaN,NaN
3,CL01402,CL,CL01,Región de Tarapacá,Camiña,"MULTIPOLYGON (((-69.31816 -19.13586, -69.31819...",CL CMA,Camiña
4,CL01403,CL,CL01,Región de Tarapacá,Colchane,"POLYGON ((-68.95021 -18.93674, -68.9502 -18.93...",CL CNE,Colchane
...,...,...,...,...,...,...,...,...
340,CL16301,CL,CL16,Región de Ñuble,San Carlos,"POLYGON ((-72.22055 -36.20812, -72.22049 -36.2...",CL SCR,San Carlos
341,CL16302,CL,CL16,Región de Ñuble,Coihueco,"POLYGON ((-71.76365 -36.4805, -71.76478 -36.48...",CL COI,Coihueco
342,CL16303,CL,CL16,Región de Ñuble,Ñiquén,"POLYGON ((-72.09907 -36.13712, -72.09972 -36.1...",CL NQN,Ñiquén
343,CL16304,CL,CL16,Región de Ñuble,San Fabián,"POLYGON ((-71.09839 -36.36997, -71.09866 -36.3...",CL SFN,San Fabián


In [38]:
gpd.GeoDataFrame(mapping, geometry="geometry", crs=chile.crs).to_file(f"{base}/cl_admin_locode.gpkg", driver="GPKG")